# Role LoRA Fine-Tuning Notebook
This notebook loads a base GPT-Neo model, applies LoRA adapters, tokenizes your dataset, and trains with the HuggingFace Trainer API.

In [ ]:
# --- Install dependencies ---
!pip install -q transformers datasets peft bitsandbytes

In [ ]:
# --- Imports ---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType
from datasets import load_dataset

In [ ]:
# --- Load dataset ---
dataset = load_dataset('json', data_files={'train': 'train.json'})['train']
texts = dataset['text']

In [ ]:
# --- Load tokenizer ---
BASE_MODEL = 'EleutherAI/gpt-neo-1.3B'  # change to your model
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# --- Tokenize dataset with padding ---
def tokenize_function(examples):
    return tokenizer(examples, padding='max_length', truncation=True, max_length=128)

tokenized = dataset.map(lambda x: tokenize_function(x['text']), batched=True)

In [ ]:
# --- Load base model and prepare for LoRA (8-bit) ---
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map='auto',
    quantization_config=bnb_config
)

# Prepare model for int8 training (necessary for LoRA)
model = prepare_model_for_int8_training(model)

In [ ]:
# --- Configure LoRA ---
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['q_proj','v_proj'],  # supported modules in GPTNeoAttention
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [ ]:
# --- Data collator ---
data_collator = DataCollatorForSeq2Seq(tokenizer, padding=True)

In [ ]:
# --- Training arguments ---
training_args = TrainingArguments(
    output_dir='role-lora',
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy='epoch',
    report_to='none'  # disables WandB logging
)

In [ ]:
# --- Initialize Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator
)

In [ ]:
# --- Train model ---
trainer.train()